# TensorFlow

In [1]:
import numpy as np
import tensorflow as tf

C:\Users\przem\AppData\Local\Continuum\anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
x = tf.Variable(3, name="x")
y = tf.Variable(4, name="y")
f = x*x*y + y + 2

Najważniejszą rzeczą do zrozumienia jest to, że ten kod w rzeczywistości nie wykonuje żadnych obliczeń, nawet jeśli wygląda na to (szczególnie na ostatniej linii). Po prostu tworzy graf obliczeniowy.

W rzeczywistości nawet zmienne nie zostały jeszcze zainicjowane. Aby wykonać ten graf, należy otworzyć sesję TensorFlow i użyć jej do zainicjowania zmiennych i wyliczenia $f$.

Sesja TensorFlow zajmuje się umieszczaniem operacji na urządzeniach takich jak procesory i GPU oraz ich uruchomieniem i przechowuje wszystkie wartości zmiennych:

In [3]:
sess = tf.Session()
sess.run(x.initializer)
sess.run(y.initializer)
result = sess.run(f)
print(result)
sess.close()

42


Ciągłe powtarzanie sesji sess.run() jest nieco uciążliwe, ale na szczęście jest lepszy sposób:

In [4]:
with tf.Session() as sess:
    x.initializer.run()
    y.initializer.run()
    result = f.eval()
    print(result)

42


Program TensorFlow dzieli się zazwyczaj na dwie części: 
    
  * pierwsza część tworzy graf obliczeniowy (nazywa się to fazą konstrukcyjną), 
  * a druga część uruchamia go (jest to faza wykonania).

# Tworzeniw grfów

Każdy utworzony węzeł jest automatycznie dodawany do domyślnego grafu:

In [9]:
x1 = tf.Variable(1)
x1.graph is tf.get_default_graph()

True

W większości przypadków jest to w porządku, ale czasami możesz chcieć zarządzać wieloma niezależnymi grafami. Możesz to zrobić, tworząc nowy graf i tymczasowo ustawiając go jako domyślny wykres wewnątrz bloku:

In [10]:
graph = tf.Graph()
with graph.as_default():
    x2 = tf.Variable(2)

x2.graph is graph

True

In [11]:
x2.graph is tf.get_default_graph()

False

W Jupyter (lub w powłoce Pythona) często używa się tych samych poleceń więcej niż raz podczas eksperymentowania. W rezultacie może pojawić się domyślny wykres zawierający wiele zduplikowanych węzłów. Jednym z rozwiązań jest ponowne uruchomienie jądra Jupytera (lub powłoki Pythona), ale wygodniejszym rozwiązaniem jest przywrócenie domyślnego wykresu poprzez uruchomienie **tf.reset_default_graph()**.

Zamiast ręcznie uruchamiać inicjalizator dla każdej zmiennej, można użyć skrótu Funkcja **global_variables_initializer()**. Zauważ, że nie wykonuje on natychmiastowej nitlizacji, lecz tworzy na wykresie węzeł, który inicjalizuje wszystkie zmienne podczas działania:

In [ ]:
init = tf.global_variables_initializer() # prepare an init node
with tf.Session() as sess:
    init.run() # actually initialize all the variables
    result = f.eval()
    print(result)

Podczas inicjalizacji węzła TensorFlow automatycznie określa zestaw węzłów, od których zależy i najpierw inicjalizuje te węzły. Rozważmy na przykład następujący kod:

In [12]:
w = tf.constant(3)
x = w + 2
y = x + 5
z = x * 3
with tf.Session() as sess:
    print(y.eval()) 
    print(z.eval())

10
15
